In [5]:
import os
import sys

In [6]:
os.chdir('/home/au643300/DataHandling/')
sys.path.append('/home/au643300/DataHandling/')

In [7]:
from dask_jobqueue import SLURMCluster
from dask.distributed import Client, as_completed
import glob
from src.data.tonetCDF import to_netcdf
import xarray as xr
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from src.interim import read_valdata
import numpy as np

In [8]:
cluster=SLURMCluster(cores=8,
                     processes=1,
                     memory="62GB",
                     queue='q64',
                     walltime='0-00:30:00',
                     local_directory='/scratch/$SLURM_JOB_ID',
                     interface='ib0',
                     extra=['--resources mem=15']
                    )
                    

In [9]:
cluster.adapt(minimum_jobs=1,maximum_jobs=16)

In [10]:
client=Client(cluster)

Her skal jeg bruge xr.open_mfdataset sammen med glob til at lave en liste over alle nc filerne, åbne dem op og tjekke om coordinaterne matcher en reference liste, list(ds.keys())

In [12]:
ref=xr.open_mfdataset("/home/au643300/NOBACKUP/test/field.0500.nc",chunks=1000,parallel=True)
ref

<xarray.Dataset>
Dimensions:  (x: 256, y: 201, z: 256)
Coordinates:
  * x        (x) float64 0.0 0.04688 0.09375 0.1406 ... 11.81 11.86 11.91 11.95
  * y        (y) float64 2.0 2.0 2.0 1.999 ... 0.00111 0.0004934 0.0001234 0.0
  * z        (z) float64 -3.0 -2.977 -2.953 -2.93 ... 2.906 2.93 2.953 2.977
    time     float64 ...
Data variables:
    u_vel    (x, y, z) float64 dask.array<chunksize=(256, 201, 256), meta=np.ndarray>
    v_vel    (x, y, z) float64 dask.array<chunksize=(256, 201, 256), meta=np.ndarray>
    w_vel    (x, y, z) float64 dask.array<chunksize=(256, 201, 256), meta=np.ndarray>
    pr1      (x, y, z) float64 dask.array<chunksize=(256, 201, 256), meta=np.ndarray>
    pr0.71   (x, y, z) float64 dask.array<chunksize=(256, 201, 256), meta=np.ndarray>
    pr0.2    (x, y, z) float64 dask.array<chunksize=(256, 201, 256), meta=np.ndarray>
    pr0.025  (x, y, z) float64 dask.array<chunksize=(256, 201, 256), meta=np.ndarray>

In [14]:
ref_key=list(ref.keys())
ref_key

['u_vel', 'v_vel', 'w_vel', 'pr1', 'pr0.71', 'pr0.2', 'pr0.025']

In [17]:
a=xr.open_mfdataset("/home/au643300/NOBACKUP/test/field.0501.nc",chunks=1000,parallel=True)
a_l=list(a.keys())

In [22]:
interim_path="/home/au643300/NOBACKUP/interim/"
files_inter=glob.glob(interim_path+'*.nc')


681

In [25]:
for file in files_inter:
    array=xr.open_mfdataset(file,chunks=1000,parallel=True)
    keys=list(array.keys())
    if keys!=ref_key:
        os.remove(file)
bad_files